In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 16.7MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 3.3MB 58.7MB/s 


In [ ]:
!pip install pennylane

     |████████████████████████████████| 460kB 18.9MB/s 


In [ ]:
!pip install datasets

     |████████████████████████████████| 204kB 18.0MB/s 
     |████████████████████████████████| 112kB 41.4MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 


In [ ]:
import time
import os
import copy
import logging
from argparse import ArgumentParser
import datetime

import datasets
from datasets import load_dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    BertForSequenceClassification,
    BertConfig,
    BertTokenizer,
    get_linear_schedule_with_warmup,
    glue_compute_metrics
)
from transformers.modeling_outputs import SequenceClassifierOutput

import pennylane as qml
from pennylane import numpy as np

In [ ]:
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("train_model.log", mode='w'),
        logging.StreamHandler()
    ]
)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

#Dum Dum

In [ ]:
step = 1e-2                 # Learning rate
batch_size = 4              # Number of samples for each training step
num_epochs = 10              # Number of training epochs
rng_seed = 42               # Seed for random number generator
start_time = time.time()    # Start of the computation timer
n_qubits = 4 # Number of qubits
q_depth = 6 # Depth of the quantum circuit (number of variational layers)
q_delta = 0.01 # Initial spread of random quantum weights

torch.manual_seed(rng_seed)

In [ ]:
train_dataset, validation_dataset= load_dataset("hatexplain", split=['train', 'validation'])

Dataset hatexplain downloaded and prepared to /root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249. Subsequent calls will reuse this data.


In [ ]:
train_dataset[0]

{'annotators': {'annotator_id': [203, 204, 233],
  'label': [0, 2, 2],
  'target': [['Hindu', 'Islam'],
   ['Hindu', 'Islam'],
   ['Hindu', 'Islam', 'Other']]},
 'id': '23107796_gab',
 'post_tokens': ['u',
  'really',
  'think',
  'i',
  'would',
  'not',
  'have',
  'been',
  'raped',
  'by',
  'feral',
  'hindu',
  'or',
  'muslim',
  'back',
  'in',
  'india',
  'or',
  'bangladesh',
  'and',
  'a',
  'neo',
  'nazi',
  'would',
  'rape',
  'me',
  'as',
  'well',
  'just',
  'to',
  'see',
  'me',
  'cry'],
 'rationales': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]]}

In [ ]:
" ".join(map(str, train_dataset[5]['post_tokens']))

'i live and work with many legal mexican immigrants who are great citizens and trump supporters they have no problem with deporting illegals maga'

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
labels = []

# For every sentence...
for sent in train_dataset:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        " ".join(map(str, sent['post_tokens'])),                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    labels.append(sent['label'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


KeyError: ignored

In [ ]:
# train_dataset, validation_dataset= load_dataset('glue', 'cola', split=['train', 'validation'])

train_dataset = train_dataset.map(lambda e: tokenizer("".join(e['post_tokens']), truncation=True, padding='max_length'), batched=True)
# validation_dataset = validation_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length'), batched=True)

# train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
# validation_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)

print(f'{len(train_dataset)} sentences in train_dataloader')
print(f'{len(validation_dataset)} sentences in validation_dataloader')

total_steps = len(train_dataloader) * num_epochs
datasets = {'train': train_dataset, 'validation': validation_dataset}
dataloaders = {'train': train_dataloader, 'validation': validation_dataloader}

KeyError: ignored

# Exp

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")

In [ ]:
model = BertForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")

In [ ]:
inputs = tokenizer("I like you. I love you", return_tensors="pt")

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs.logits

tensor([[-1.7712,  1.2486, -0.1811]], grad_fn=<AddmmBackward>)

In [ ]:
torch.nn.functional.softmax(outputs.logits,dim=1)

tensor([[0.0379, 0.7763, 0.1858]], grad_fn=<SoftmaxBackward>)

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# data clean

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv")

In [ ]:
# data = data.reset_index()
data = data.drop(['Unnamed: 0'],axis=1)

In [ ]:
def classmap(n):
    if (n == 2):
        return 1
    if (n == 1):
        return 2
    return 0

In [ ]:
data['new_class'] = pd.DataFrame(map(classmap,data['class']))

In [ ]:
data.at[5,"tweet"]

'!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"'

In [ ]:
!pip install demoji

In [ ]:
import demoji

In [ ]:
demoji.download_codes()

... OK (Got response in 0.14 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [ ]:
data['cleaned_tweet_text_emoji'] = data.tweet.map(lambda x: demoji.replace_with_desc(x, sep = " "))

In [ ]:
!pip install ekphrasis

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 184kB 25.9MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=8461403e17e5a4b5946e03c932d33c1097fecd492c6dd76a85710dff6693c26f
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=b16368c6eeda9063641d531ba9dec09e64f0358a879a972ac68c646eb6f776c9
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ekphrasis ftfy


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
import string 
import re
import spacy
nlp2 = spacy.load('en_core_web_sm')
from spacy.symbols import ORTH,NORM,LEMMA
import string 
from spacy.lang.char_classes import LIST_PUNCT, LIST_ELLIPSES, LIST_QUOTES, LIST_CURRENCY
from spacy.lang.char_classes import LIST_ICONS, HYPHENS, CURRENCY, UNITS
from spacy.lang.char_classes import CONCAT_QUOTES, ALPHA_LOWER, ALPHA_UPPER, ALPHA, PUNCT
from spacy.util import compile_infix_regex, compile_prefix_regex, compile_suffix_regex

In [ ]:
##### text preprocessor for ekphrasis
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    fix_html=True,  # fix HTML tokens
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    #corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example', 'smartphones':'smartphone'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does','Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_contractions(text):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")

    for word in contraction_mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+contraction_mapping[word]+"")

    for p in punct_mapping:
      text = text.replace(p, punct_mapping[p])

    for p in punct:
      text = text.replace(p,' ')

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy.He is good" => "he is a boy .He is good" so the it doesn't become 
    # he is a boyHe is good
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # Remove extra space
    text = re.sub(r' +', ' ', text)

    return text

In [ ]:
def remove_urls(text):
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'', text)

def clean_reserved(text):
  text = text.replace('@', ' ').replace('#',' ').replace('_',' ').replace('RT', ' ').replace('rt', ' ')
  return text
  
def camel_case_split(identifier):
  matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
  return " ".join([m.group(0) for m in matches])

In [ ]:
def preprocess(text):
    remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','<user>','<date>', '<number>','\'','s']
    word_list=text_processor.pre_process_doc(text)
    word_list=list(filter(lambda a: a not in remove_words, word_list)) 
    word_list=[token for token in word_list if token not in string.punctuation]
    tw1 = " ".join(word_list)
    # tw1 = text
    tw1 = remove_urls(tw1)
    tw1 = clean_reserved(tw1)
    tw1 = camel_case_split(tw1)
    tw1 = tw1.lower()
    tw1 = clean_contractions(tw1)
    return tw1

In [ ]:
preprocess(data.at[546,"tweet"])

'searching the incredible story of white man quest to find the ingredients necessary to make caribbean jerked chicken in redneck land'

In [ ]:
data['cleaned_tweet'] = data['cleaned_tweet_text_emoji'].apply(preprocess)

In [ ]:
data.at[4828,"cleaned_tweet"]

'so trash'

In [ ]:
for x in range(len(data["cleaned_tweet"])):
    l = len(data.at[x,"cleaned_tweet"])
    if (l<=2):
        print(x, data.at[x,"cleaned_tweet"])
        data.drop([x],inplace=True)

In [ ]:
# data.drop([4828, 6098],inplace=True)

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head()

,count,hate_speech,offensive_language,neither,class,tweet,new_class,cleaned_tweet_text_emoji,cleaned_tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,1,!!! RT @mayasolovely: As a woman you shouldn't...,as a woman you should not complain about clea...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,2,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dats cold tyga dwn bad for cuffin dat hoe...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg you ever fuck a bitch and she sta to cry...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,2,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,she look like a tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,2,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,the shit you hear about me might be true or i...


In [ ]:
data = data.drop(columns=["cleaned_tweet_text_emoji"],axis=1)

In [ ]:
data.to_csv("preprocess_tweets.csv",index=False)

In [ ]:
df = pd.read_csv("preprocess_tweets.csv")

In [ ]:
df['new_class'].value_counts()

2    19189
1     4162
0     1430
Name: new_class, dtype: int64

# Bert Trial

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

In [ ]:
traindf, valdf = train_test_split(df, test_size=0.1)

In [ ]:
traindf.head()

,count,hate_speech,offensive_language,neither,class,tweet,new_class,cleaned_tweet
10202,3,1,2,0,1,I don't fuck with you iffy bitches/niggas. Y'a...,2,i do not fuck with you iffy bitches niggas you...
11416,3,1,2,0,1,I'm thinking he has low level OCD of some sort...,2,i am thinking he has low level ocd of some so ...
2812,3,0,3,0,1,@CenaSux43 You/ @TheBrianMo brought it up firs...,2,you brought it up first with the allusion that...
5398,3,1,2,0,1,@_elennamichelle only when ur around me &#1285...,2,only when ur around me gotta keep dat scent to...
12072,3,0,1,2,2,Jindal seemed to be having an out-of-body expe...,1,jindal seemed to be having an out of body expe...


In [ ]:
print('{:>5,} training samples'.format(len(traindf)))
print('{:>5,} validation samples'.format(len(valdf)))

22,302 training samples
2,479 validation samples


In [ ]:
def makedataset(dataf):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in dataf['cleaned_tweet']:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
    
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(dataf['new_class'].values)
    dataset = TensorDataset(input_ids, attention_masks, labels)
    return dataset

In [ ]:
train_dataset = makedataset(traindf)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
val_dataset = makedataset(valdf)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
class_sample_count_tdf = np.array(
    [len(np.where(traindf['new_class'] == t)[0]) for t in np.unique(traindf['new_class'].values)])

In [ ]:
class_sample_count_tdf.requires_grad = False

In [ ]:
weight_tdf = 1. / class_sample_count_tdf

In [ ]:
samples_weight_tdf = np.array([weight_tdf[t] for t in traindf['new_class'].values])

In [ ]:
samples_weight_tdf = torch.from_numpy(samples_weight_tdf)
samples_weigth_tdf = samples_weight_tdf.double()

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

In [ ]:
sampler_tdf = WeightedRandomSampler(samples_weight_tdf, len(samples_weight_tdf)) #weighted sampler to tackle imbalanced classes

In [ ]:
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = sampler_tdf, # Select batches using weights as classes are imbalanced
            batch_size = batch_size # Trains with this batch size.
        )

In [ ]:
# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain",
                                                      num_labels = 3,
                                                      output_attentions = False, # Whether the model returns attentions weights.
                                                      output_hidden_states = False, # Whether the model returns all hidden-states.
                                                      )

In [ ]:
inputs = tokenizer("I like you. I love you", return_tensors="pt")

In [ ]:
outputs = model(**inputs)